### 第一个普通的tensorflow demo启动过程如下：


In [3]:
import tensorflow as tf
hello = tf.constant('Hello ,TF')
sess = tf.Session()
print(sess.run(hello))

b'Hello ,TF'


## 第一个小Demo--- 生成三维数据

In [14]:


import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化变量
init = tf.initialize_all_variables()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

# 得到最佳拟合结果 W: [[0.100  0.200]], b: [0.300]

0 [[ 0.19164234 -0.11989146]] [ 0.90815353]
20 [[ 0.06188139  0.0751335 ]] [ 0.38528293]
40 [[ 0.08628521  0.16677572]] [ 0.32457691]
60 [[ 0.0956113   0.19095987]] [ 0.30703396]
80 [[ 0.09866986  0.19750349]] [ 0.30200484]
100 [[ 0.09960821  0.19930395]] [ 0.30057001]
120 [[ 0.09988642  0.19980475]] [ 0.30016184]
140 [[ 0.09996738  0.19994505]] [ 0.30004588]
160 [[ 0.09999068  0.19998451]] [ 0.30001301]
180 [[ 0.09999735  0.19999562]] [ 0.30000371]
200 [[ 0.09999924  0.19999875]] [ 0.30000106]


# TF 基本使用

> TF 特点：
> * 本质上是： 一个编程系统
> * 使用图（graph）来表示计算任务
> * 在 被称为 会话（Session）的上下文（context）中 执行图
> * 使用`tensor` 表示数据, 每一个`Tensor`表示一个类型化的多维数组。
> * 图中的节点称为`op(operation)`，一个`op` 执行计算，获得0或多个`Tensor`
> * 通过`变量variable`维护状态
> * 使用 feed 和 fetch 可以 为 任意的操作（arbitrary operation） 赋值或者从其中获取数据

### TF运行过程
一个TF图描述一个计算的过程，图在 会话 中被启动，会话将图的`op`分发到`CPU`,`GPU` 等设备中，并同时提供执行`op`的方法。 而这些方法执行完毕之后，将产生的`tensor`返回。 在`Python`中， 返回的`tensor`是`numpy`的`ndarray`对象（也就是N维矩阵）

TF程序一般被组织成两个阶段：

**(1) 构建阶段**
* `op`的执行步骤被描述成一个图


In [15]:
import tensorflow as tf

# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点
# 加到默认图中.
#
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

**(2) 执行阶段**
* 使用`session`执行`graph`中的`op`

In [16]:
# 启动默认图
sess = tf.Session()
result = sess.run(product)
print(result)

# task finished, session close.
sess.close()

[[ 12.]]


**TF将图形定义转换为分布式执行操作，so that，充分利用可用的计算资源（如CPU/GPU）**
一般你不需要显式指定使用 CPU 还是 GPU, TensorFlow 能自动检测. 如果检测到 GPU, TensorFlow 会尽可能地利用找到的第一个 GPU 来执行操作.

**如果你有足够多的GPU，那就更好啦**
如果机器上有超过一个可用的 GPU, 除第一个外的其它 GPU 默认是不参与计算的. 为了让 TensorFlow 使用这些 GPU, 你必须将 `op` 明确指派给它们执行. `with...Device` 语句用来指派特定的 CPU 或 GPU 执行操作

In [ ]:
# with tf.Session() as sess:
#   with tf.device("/gpu:1"):
#     matrix1 = tf.constant([[3., 3.]])
#     matrix2 = tf.constant([[2.],[2.]])
#     product = tf.matmul(matrix1, matrix2)
#     ...

#### TF 支持的编程语言
* C
* C++
* Python （易用）
> 三种语言的 会话库（`session libraries`）是一致的。

In [17]:
import numpy as np
arrA = np.array([[0,1]])
arrB = np.array([[2],[3]])
result = np.dot(arrA, arrB)
print(result)

[[3]]


## 交互式使用
---
目的：方便使用如`IPython`等的交互环境, 并且，避免使用一个变量来持有会话。
怎么做:
* `InterativeSession` insead of `Session`
* `Tensor.eval()`+ `Operation.run()` instead of `Session.run()`

In [38]:
import tensorflow as tf
#  进入一个交互式 TF会话
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器 initializer op 的 run() 方法初始化 ‘x’
x.initializer.run()

# tf.sub() 表示减法，op 减 op
res = tf.subtract(x,a)
print(res.eval())

[-2. -1.]


## 变量
Variable  用于维护图执行过程中的状态信息。

In [40]:
#  例子：计数器

# -----------------------------------------------------------------
# 构建阶段

state = tf.Variable(0, name="counter")

# 创建OP， 作用：使得 state 增加 1
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 启动图 之后， 变量需要经过‘初始化’ ，首先需要增加一个 `初始化`op 进到图中
init_op = tf.initialize_all_variables()

# -----------------------------------------------------------------
# 执行阶段

#  计数开始
with tf.Session() as sess:
    sess.run(init_op)
    # 打印 state 初始值
    print(sess.run(state))
    # 运行op ,update `state`
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))


0
1
2
3


通常会将一个统计模型中的参数表示为一组变量. 例如, 你可以将一个神经网络的权重作为某个变量存储在一个 tensor 中. 在训练过程中, 通过重复运行训练图, 更新这个 tensor.

### Fetch
Session 对象 run() 是执行图，
tensor  你可以理解为是：一个数据结构（一个对象，这个对象可以装矩阵等数据）

所以，如果想拿到run()的结果，就用一个tensor去装 结果回来。

In [48]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

# 所有变量都是 tensor
print('input1\n', input1)
print('state\n', state)
print('乘法mul\n', mul)

with tf.Session() as sess:
    # 同时获取 两个 op 执行的结果， 放到 一个 tensor `result` 中
    result = sess.run([mul, intermed])
    print('执行加法 和 乘法，得到结果：', result)

input1
 Tensor("Const_45:0", shape=(), dtype=float32)
state
 <tf.Variable 'counter_5:0' shape=() dtype=int32_ref>
乘法mul
 Tensor("Mul_6:0", shape=(), dtype=float32)
执行加法 和 乘法，得到结果： [21.0, 7.0]


### Feed
TensorFlow提供了Feed机制：可以临时替代图中的任意操作中的 tensor ，可以对图中任何操作提交补丁，直接插入一个 tensor。

> feed 使用一个tensor 值  临时替换一个操作的输出结果。你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失. 最常见的用例是将某些特殊的操作指定为 "feed" 操作, 标记的方法是使用 tf.placeholder() 为这些操作创建占位符

In [51]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.],input2:[2.]}))
    

[array([ 14.], dtype=float32)]
